<hr>

<table style="width:100%">
  <tr>
    <th><img align="center" src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/UNAL_Aplicación_Medell%C3%ADn.svg/1280px-UNAL_Aplicación_Medell%C3%ADn.svg.png" width="300"/></th>
    <th><img align="center" src="http://www.redttu.edu.co/es/wp-content/uploads/2016/01/itm.png" width="300"/> </th> 
    <th><img align="center" src="https://www.cienciasdelaadministracion.uns.edu.ar/wp-content/themes/enlighten-pro/images/logo-uns-horizontal.png" width="300"/></th>
  </tr>
</table>


<hr>

#### Pedro Atencio Ortiz - 2019 (pedroatencio@itm.edu.co)

# Módulo 2

En este notebook abordaremos los siguientes tópicos:

1. Primera red en Keras 
2. Ensamble de la red: a) Construcción como lista b) Agregación de capas (model.add) b) "Cableado" manual.
3. Compilación
4. Preparación del dataset
    1. Estructura del dataset.
    2. Train / Test (dataset split).
    3. K-Fold.
5. Entrenamiento y validación.
6. Ejemplo completo.

In [ ]:
# Funciones utilitarias

import numpy as np
import sklearn
from sklearn import datasets
import matplotlib.pyplot as plt

import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

def generate_data(data_type, noise=0.2):
    
    np.random.seed(0)
    if data_type == 'moons':
        X, Y = datasets.make_moons(200, noise=noise)
    elif data_type == 'circles':
        X, Y = sklearn.datasets.make_circles(200, noise=noise)
    elif data_type == 'blobs':
        X, Y = sklearn.datasets.make_blobs(centers=2, cluster_std=noise)
    return X, Y

def visualize_model(model, X, Y, output='truncate'):
    XT = np.copy(X)
    # Set min and max values and give it some padding
    x_min, x_max = XT[:, 0].min() - .5, XT[:, 0].max() + .5
    y_min, y_max = XT[:, 1].min() - .5, XT[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    if(output=='truncate'):
        Z = np.round(model.predict(np.c_[xx.ravel(), yy.ravel()]))
    elif(output=='same'):
        Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    else:
        print("output param must be either truncate or same")
        return False

    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.figure(figsize=(7,5))
    plt.contourf(xx, yy, Z, cmap=plt.cm.bone)

    color = ['blue' if y == 1 else 'red' for y in np.squeeze(Y)]
    plt.scatter(X[:,0], X[:,1], color=color)

    plt.show()

<hr>

## 1. Primera red en Keras

Para este ejercicio, utilizaremos Keras para construir la misma red neuronal del módulo anterior e igualmente entrenaremos la misma mediante los optimizadores de la librería.

<img align="center" src="https://github.com/psatencio/intro_keras/blob/master/figures/layered_net.png?raw=true" width="500"/>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop

In [ ]:
# Dataset
X = np.array([[0,0],[0,1],[1,0],[1,1]])
Y = np.array([[0], [1], [1], [0]])

nx = X.shape[0]
m = len(X)

En keras, la capa de tipo __Dense__ equivale a una capa totalmente conectada (fully connected), es decir, una capa que recibe como entrada todas las salidas de la capa anterior, y entrega todas sus salidas a la siguiente capa.

In [ ]:
#tf.keras.backend.clear_session() #borra el grafo de la sesion. Util cuando creamos muchos modelos en una sesion.

# Red neuronal
model = Sequential()
model.add(Dense(units=3, input_dim=2, activation='sigmoid', use_bias=True)) #capa 1. La dimensionalidad de la entrada solo se define para la primera capa
model.add(Dense(units=1, activation='sigmoid', use_bias=True)) #capa 2

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy', # funcion loss
              metrics=['accuracy']) # metricas complementarias

In [ ]:
history = model.fit(X,Y, epochs=3000, verbose=0)

print("Error (Loss) final: %.4E"%(np.array(history.history['loss'][-1:]))) #Error final de la lista de errores
print("Precision (Accuracy) final: %.2f"%(np.array(history.history['acc'][-1:])))

plt.plot(history.history['loss'])
plt.plot(history.history['acc'])
plt.show()

In [ ]:
visualize_model(model, X, Y, output='truncate')

<hr>

### Trabajemos

<br>

<font size=4>

1. Utilicemos lo implementado al momento para clasificar los problemas __'moons'__ y __'circles'__ (ver figura siguiente), y probar:

<br>

<ul>

<li>Pruebe agregando más neuronas en la capa 1.</li>
<li>Pruebe agregando más capas a la red.</li>
<li>Pruebe distintas configuraciones $\alpha$ (learning rate).</li>
<li>Pruebe distintos valores para el número de épocas.</li>
</ul>

</font>

In [ ]:
X, Y = generate_data('moons', 0.1)

color = ['blue' if y == 1 else 'red' for y in np.squeeze(Y)] # una lista para darle color a las clases

plt.figure(figsize=(7,5))
plt.scatter(X[:,0], X[:,1], color=color)
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.grid()

plt.show()

<hr>

## 2. Ensamble de la red

Dependiendo del nivel de especificidad deseado para la arquitectura de la red neuronal, podemos utilizar diferentes formas de construirla (ver figura).

<img align="center" src="https://github.com/psatencio/intro_keras/blob/master/figures/build_strategies.png?raw=true" width="500"/>

En la figura anterior, podemos observar tres formas de construir una misma red neuronal:

1. Ingresando las capas como una lista (A).
2. Agregando las capas mediante el módulo add() (B).
3. Configurando el gráfo de cómputo (C).

A continuación, implementemos cada caso.

In [ ]:
# Como lista
modelA = Sequential([Dense(units=2, input_dim=2, activation='sigmoid'),
                   Dense(units=1, activation='sigmoid')])

In [ ]:
# Utilizando el modulo add (agregar)
modelB = Sequential()
modelB.add(Dense(units=2, input_dim=2, activation='sigmoid'))
modelB.add(Dense(units=1, activation='sigmoid'))

In [ ]:
# "Cableando" manualmente las capas

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

x = Input(shape=(2,))
a1 = Dense(units=2, activation='sigmoid')(x)
a2 = Dense(units=1, activation='sigmoid')(a1)

modelC = Model(inputs=x, outputs=a2)

In [ ]:
modelC.compile(optimizer='rmsprop',
              loss='binary_crossentropy', # funcion loss
              metrics=['accuracy']) # metricas complementarias

In [ ]:
history = modelC.fit(X,Y, epochs=200, verbose=0)

print("Error (Loss) final: %.4E"%(np.array(history.history['loss'][-1:]))) #Error final de la lista de errores
print("Precision (Accuracy) final: %.2f"%(np.array(history.history['acc'][-1:])))

#plt.plot(history.history['loss'])
#plt.plot(history.history['acc'])
#plt.show()

In [ ]:
visualize_model(modelC, X, Y)

<hr>

### Discutamos

<br>

<font size=4>

- En qué situaciones conviene una aproximación u otra?

</font>

<hr>

## 3. Compilación

La compilación permite definir los elementos del entrenamiento del modelo, particularmente, permite configurar:

- <a href="https://keras.io/optimizers/">El optimizador</a>.
- <a href="https://keras.io/losses/">La función de error</a>.
- <a href="https://keras.io/metrics/">Las métricas de evaluación</a>.

In [ ]:
#Dataset

X, Y = generate_data('circles', 0.1)

color = ['blue' if y == 1 else 'red' for y in np.squeeze(Y)] # una lista para darle color a las clases

plt.figure(figsize=(7,5))
plt.scatter(X[:,0], X[:,1], color=color)
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.grid()

plt.show()

In [ ]:
tf.keras.backend.clear_session() #borra el grafo de la sesion. Util cuando creamos muchos modelos en una sesion.

#Construccion de la red
model = Sequential([Dense(units=5, input_shape=(2, ) , activation='sigmoid'),
                   Dense(units=1, activation='sigmoid')])

<hr>

Podemos especificar con alto nivel de detalle cada elemento de la compilacion

In [ ]:
optimizador = tf.keras.optimizers.RMSprop(lr=0.1) #objeto optimizador de tipo RMSprop
error = tf.keras.losses.binary_crossentropy
area_roc = tf.keras.metrics.AUC()

También podemos crear una métrica propia. Para ello debemos crear una función con los parámetros de entrada $(y\_true, y\_pred)$, donde $y\_true$ es un tensor de las categorías reales de $X$ y $y\_pred$, un tensor con las predicciones realizadas por el modelo.

Por otra parte, las operaciones al interior de la función deben estar construidas utilizando las funciones del backend de Keras __tensorflow.keras.backend__ o de tensorflow.

__Nota__: En caso de ser necesario crear una métrica que no dependa exclusivamente de $(y\_true, y\_pred)$, es necesario utilizar __clausura__ de funciones (<a href="https://es.wikipedia.org/wiki/Clausura_(informática)">function closure</a>). Para ahondar en el tema, revisar el siguiente <a href="https://towardsdatascience.com/advanced-keras-constructing-complex-custom-losses-and-metrics-c07ca130a618">enlace</a>.

Supongamos que queremos mostrar la precisión entre 0 y 100:

In [ ]:
import tensorflow.keras.backend as K

def metrica_propia(y_true, y_pred):
    '''
    Metrica de prueba. Devuelve el accuracy en porcentaje.
    '''
    y = K.round(y_true)
    a = K.round(y_pred)
    
    return (K.constant(1) - K.mean(K.abs(y-a))) * K.constant(100)

Finalmente, utilizamos la función __compile__ predefinida en los modelos de Keras y enviamos los siguientes argumentos a los parámetros en esta forma:

- __optimizer__ : Optimizador (instancia) o string predefinido. <a href="https://keras.io/optimizers/">tensorflow.keras.optimizers</a>.
- __loss__: Función de error (función) o string predefinido. <a href="">tensorflow.keras.losses</a>
- __metrics__: <font color="red">Lista</font> de funciones de métricas, o de strings de métricas predefinidas. <a href="https://keras.io/metrics/">tensorflow.keras.metrics</a>

In [ ]:
model.compile(optimizer=optimizador, loss=error, metrics=['acc', area_roc, metrica_propia])

history = model.fit(X,Y, epochs=200, verbose=0)

In [ ]:
print(history.history.keys())

plt.figure(figsize=(10,5))

plt.plot(history.history['loss'])
plt.plot(history.history['acc'])
plt.plot(history.history['auc'])

plt.legend(['loss', 'Accuracy','Area ROC'])

In [ ]:
print("Error (Loss) final: %.4E"%(np.array(history.history['loss'][-1:]))) #Error final de la lista de errores

print("Precision (Accuracy) final: %.2f"%(np.array(history.history['acc'][-1:])))
print("Area debajo de la curva ROC final: %.2f"%(np.array(history.history['auc'][-1:])))
print("Precision entre 0-100 final: %.2f"%(np.array(history.history['metrica_propia'][-1:])))

In [ ]:
visualize_model(model, X, Y, output='same')

<hr>

## 4. Preparación del dataset

En cualquier aproximacion de aprendizaje de maquina es necesario validar el modelo utilizando alguna estrategia de separacion de datos, por ejemplo, split o k-fold (ver figura).

En esta seccion trabajaremos sobre estas aproximaciones utilizando arreglos de Numpy para manipular los datasets.

__Nota:__ Al trabajar con datasets con grandes cantidades de ejemplos, resulta inviable en muchos casos utilizar arreglos de Numpy que requieren cargarse en memoria. Para ello es posible utilizar <a href="https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c">generadores</a>.

<img align="center" src="https://github.com/psatencio/intro_keras/blob/master/figures/train_test.png?raw=true" width="500"/>